In [ ]:
from defillama2alpha import DefiLlama
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.style.use("fivethirtyeight")

# Makes graphs crisp. Only use if you don't have a lot of points/lines on your graph.
%config InlineBackend.figure_formats = ['svg', 'retina'] # use svg, then retina if svg is not supported

In [ ]:
def human_format_dollar_or_num(dollar=False, decimals=0):
    """ 
    Return a function for formatting matplot axis numbers human friendly. 

    Parameters
    ----------
    dollar: logical
        If True, will add $ in front of the numbers.
    decimals: int
        Number of decimals to display.
    """

    base_fmt = '%.{}f%s'.format(decimals)
    if dollar:
        base_fmt = '$' + base_fmt

    def human_format(num, pos): # pos is necessary as it'll be used by matplotlib
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        return base_fmt % (num, ['', 'K', 'M', 'B', 'T', 'P'][magnitude])
    return human_format

## Create a DefiLlama instance

In [ ]:
obj = DefiLlama()

## Get historical DeFi TVL on all chains

In [ ]:
df = obj.get_defi_hist_tvl()
df.tail()

In [ ]:
ax = df.tvl.plot()
dollar_formatter = FuncFormatter(human_format_dollar_or_num(dollar=True, decimals=0))
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("DeFi TVL");

## Get historical TVL of a chain

In [ ]:
df = obj.get_chain_hist_tvl('Arbitrum')
df.tail()

In [ ]:
ax = df.tvl.plot()
dollar_formatter = FuncFormatter(human_format_dollar_or_num(dollar=True, decimals=1))
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("Arbitrum TVL");

## Get current TVL of all chains

In [ ]:
df = obj.get_chains_curr_tvl()

# print top 10 chains by tvl
df = df.sort_values('tvl', ascending=False) 
df.head(10)

## Get current TVL of a protocol

In [ ]:
print('Current TVL of Uniswap:', obj.get_protocol_curr_tvl('uniswap'))
print('Current TVL of Aave:', obj.get_protocol_curr_tvl('aave'))

In [ ]:
obj.get_protocol_curr_tvl('gmx')

In [ ]:
obj.get_protocol_curr_tvl('premia')

## Get current TVL of a protocol broken down by chain

In [ ]:
df = obj.get_protocol_curr_tvl_by_chain('gmx')
df

In [ ]:
df = obj.get_protocol_curr_tvl_by_chain('premia')
df

In [ ]:
# let's get AAVE's current TVL
obj.get_protocol_curr_tvl_by_chain('aave')

## Get historical TVL of a protocol by chain

In [ ]:
dd = obj.get_protocol_hist_tvl_by_chain('gmx')

In [ ]:
ax = dd['Arbitrum'].tvl.plot()
dollar_formatter = FuncFormatter(human_format_dollar_or_num(dollar=True, decimals=0))
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("GMX TVL on Arbitrum, excluding staked GMX/esGMX");

In [ ]:
ax = dd['Arbitrum-staking'].tvl.plot()
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("staked GMX/esGMX TVL on Arbitrum");

In [ ]:
ax = dd['Avalanche'].tvl.plot()
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("GMX TVL on Avalanche, excluding staked GMX/esGMX");

In [ ]:
ax = dd['Avalanche-staking'].tvl.plot()
ax.yaxis.set_major_formatter(dollar_formatter)
plt.ylabel("USD")
plt.title("staked GMX/esGMX TVL on Avalanche");

## Get fundamentals for all protocols

In [ ]:
df = obj.get_protocols_fundamentals()
df = df.sort_values('tvl', ascending=False, ignore_index=True)
df.head(10)

In [ ]:
# find the 5 protocols with the biggest % jump in TVL in last 7 days
df.sort_values('change_7d', ascending=False, ignore_index=True).head(5)

In [ ]:
# derive other metrics
ha = (df.loc[:, ['name', 'category', 'tvl', 'mcap']]#'fdv'
        .assign(mcap_per_tvl = df.mcap / df.tvl) #fdv_per_tvl = df.fdv / df.tvl)
     )
ha          

In [ ]:
# aggregate by category
ha.groupby('category')[['tvl','mcap']].sum()#'fdv'